# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
#from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

#from azureml.pipeline.steps import AutoMLStep
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.sklearn import SKLearn

from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.parameter_expressions import choice

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.13.0


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()

key = "Heart-Failure"

if key in ws.datasets.keys(): 
        dataset = ws.datasets[key] 


experiment_name = 'heart_failure_HyperDrive'

experiment=Experiment(ws, experiment_name)


In [ ]:
#import os
project_folder = './SkLearn'
#os.makedirs(project_folder, exist_ok=True)

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpucluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
from azureml.train.hyperdrive import BanditPolicy
early_termination_policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)


#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling( {
    "--kernel": choice('linear', 'rbf', 'poly', 'sigmoid'),
    "--penalty": choice(0.5, 1, 1.5)
    })

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(source_directory=project_folder, 
                    script_params={ '--kernel': 'linear', '--penalty': 1.0 },
                    compute_target=cpu_cluster,
                    entry_script='train.py',
                    pip_packages=['joblib==0.13.2']
                   )

hyperdrive_run_config =  HyperDriveConfig(estimator=estimator,
                                     hyperparameter_sampling=param_sampling, 
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=12,
                                     max_concurrent_runs=4)

WARNING - You have specified to install packages in your run. Note that you have overridden Azure ML's installation of the following packages: ['joblib']. We cannot guarantee image build will succeed.


In [ ]:
#TODO: Submit your experiment
remote_run = experiment.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_run = remote_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n Best Params:',parameter_values)

Best Run Id:  HD_6d3a6dee-3fbd-4eac-acb6-5b516ac6e7c0_2

 Accuracy: 0.8133333333333334

 Best Params: ['--kernel', 'sigmoid', '--penalty', '1.5']


In [ ]:
#TODO: Save the best model

print(best_run.get_file_names())

model = best_run.register_model(model_name='best-trained-model', model_path='outputs/model.joblib')

['azureml-logs/55_azureml-execution-tvmps_1feadac1839672147ca24d400e5f505ec660cfa32b22dc60a732d6dd6742b45a_d.txt', 'azureml-logs/65_job_prep-tvmps_1feadac1839672147ca24d400e5f505ec660cfa32b22dc60a732d6dd6742b45a_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_1feadac1839672147ca24d400e5f505ec660cfa32b22dc60a732d6dd6742b45a_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/110_azureml.log', 'logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/engine_spans_l_f6b37010-20db-441f-b160-2cc4e46c414f.jsonl', 'logs/azureml/dataprep/python_span_l_f6b37010-20db-441f-b160-2cc4e46c414f.jsonl', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/model.joblib']


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
from azureml.core.model import Model
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core import Workspace, Environment


#model = Model.register(model_path="best-trained-model.pkl",
#                       model_name="best-trained-model",
#                       workspace = ws)



service_name = 'my-custom-env-service'


env = Environment.get(workspace=ws, name="AzureML-Minimal")

inference_config = InferenceConfig(entry_script='score.py', environment=env)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=4)

service = model.deploy(workspace=ws,
                          name=service_name,
                          models=[model],
                          inference_config=inference_config,
                          deployment_config=aci_config)
                       
service.wait_for_deployment(show_output=True)

# enable application insight
service.update(enable_app_insights=True)

print(service.state)
print("scoring URI: " + service.scoring_uri)

Running................................

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
import requests
import json

scoring_uri = service.scoring_uri
headers = {'Content-Type':'application/json'}

test_data = json.dumps({'age': 75, 'anaemia': 0, 'creatinine_phosphokinase': 582, 'diabetes': 0, 'ejection_fraction':20, 'high_blood_pressure': 1, 'platelets': 265000, 'serum_creatinine': 1.9, 'serum_sodium': 130, 'sex':1, 'smoking':0 })

response = requests.post(scoring_uri, data=test_data, headers=headers)
print(response.status_code)
print(response.elapsed)
print(response.json())

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
logs = service.get_logs()

for line in logs.split('\n'):
    print(line)